In [3]:
library(fastDummies)
library(cplm)
library(tweedie)
library(statmod)
library(HDtweedie)
library(TDboost)
library(mgcv)
library(MASS)
library(glmnet)

Loading required package: coda

Loading required package: Matrix

Loading required package: splines

Loading required package: lattice

Loaded TDboost 1.2


Loading required package: nlme


Attaching package: 'nlme'


The following objects are masked from 'package:cplm':

    fixef, ranef, VarCorr


This is mgcv 1.8-38. For overview type 'help("mgcv-package")'.

Loaded glmnet 4.1-2



In [37]:
beta_num<- rep(0.3,9)
mu<-exp(design_x%*%beta_num)
y_sim<- rTweedie(mu,p=1.7,phi=9)

In [38]:
length(which(y_sim==0))

[1] 3105

In [39]:
summary(y_sim)

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
   0.000    0.000    0.000   13.282    1.123 7440.451 

In [40]:
sim_full_train<-cbind(design_x_train,y_sim)

In [41]:
design_x_train$y=y_sim

In [42]:
sam <- sample(rep(1:2,len=5000))
sim_train<-sim_full_train[sam==1,]
sim_test<-sim_full_train[sam==2,]

In [43]:
length(which(sim_train$y==0))
length(which(sim_test$y==0))

[1] 1548

[1] 1557

In [44]:
design_x_strain<-design_x_train[sam==1,]
design_x_stest<-design_x_train[sam==2,]
design_x_strain$y=sim_train$y_sim

In [45]:
true_fit<- glm(y~.+x1*x2*x4,data=design_x_strain,family=tweedie(link.power=0,var.power=1.7))
summary(true_fit)
mean(residuals(true_fit)^2)


Call:
glm(formula = y ~ . + x1 * x2 * x4, family = tweedie(link.power = 0, 
    var.power = 1.7), data = design_x_strain)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-8.4205  -2.7855  -2.4277  -0.8794  11.7444  

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.26859    0.17005   1.579  0.11436    
x12          0.30777    0.18881   1.630  0.10321    
x2           0.27333    0.15059   1.815  0.06963 .  
x3           0.30693    0.05944   5.164 2.61e-07 ***
x4           0.31626    0.10548   2.998  0.00274 ** 
x12:x2       0.44195    0.18675   2.367  0.01803 *  
x12:x4       0.21599    0.12688   1.702  0.08881 .  
x2:x4        0.31756    0.09971   3.185  0.00147 ** 
x12:x2:x4    0.23217    0.11727   1.980  0.04784 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Tweedie family taken to be 10.11432)

    Null deviance: 36207  on 2499  degrees of freedom
Residual deviance: 17586  on 249

[1] 7.034569

In [46]:
fit1<-glm(y~.,data=design_x_strain,family=tweedie(link.power=0,var.power=1.7),control=list(maxit=500))
summary(fit1)
mean(residuals(fit1)^2)


Call:
glm(formula = y ~ ., family = tweedie(link.power = 0, var.power = 1.7), 
    data = design_x_strain, control = list(maxit = 500))

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-6.294  -3.014  -2.437  -1.012  18.083  

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -0.09748    0.22520  -0.433 0.665162    
x12          0.87784    0.18643   4.709 2.63e-06 ***
x2           1.09475    0.09011  12.149  < 2e-16 ***
x3           0.33560    0.09500   3.532 0.000419 ***
x4           0.74848    0.09430   7.937 3.10e-15 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Tweedie family taken to be 27.89925)

    Null deviance: 36207  on 2499  degrees of freedom
Residual deviance: 19445  on 2495  degrees of freedom
AIC: NA

Number of Fisher Scoring iterations: 52


[1] 7.77803

In [48]:
autocross<-design_x_strain[,1:4]
summary(autocross)
ytrain= design_x_strain$y

 x1             x2                  x3                x4         
 1: 985   Min.   :-4.130126   Min.   :-4.2001   Min.   :-2.5243  
 2:1515   1st Qu.:-0.664392   1st Qu.:-1.6615   1st Qu.: 0.3727  
          Median : 0.007939   Median :-0.9617   Median : 1.0504  
          Mean   :-0.004948   Mean   :-0.9897   Mean   : 1.0281  
          3rd Qu.: 0.684042   3rd Qu.:-0.3411   3rd Qu.: 1.6989  
          Max.   : 2.890667   Max.   : 2.9720   Max.   : 4.2413  

In [49]:
result_round1=matrix(nrow=dim(autocross)[2],ncol=dim(autocross)[2])
cat_coef=list()
cat_coef_names=c()
for(i in 2:(dim(autocross)[2])-1){
    for(j in (i+1):(dim(autocross)[2])){
        if (is.numeric(autocross[,i])&&is.numeric(autocross[,j])){
            tmp_x <- autocross[,i]*autocross[,j]
            tmp_fit<-glm(ytrain~tmp_x+offset(log(fitted(fit1)))
                   ,family=tweedie(link.power=0,var.power=1.7),control=list(maxit=500))
            result_round1[i,j]=mean(residuals(tmp_fit)^2)
        }
        else{
            tmp_x<-model.matrix(~autocross[,i]*autocross[,j])
            tmp_x<-as.matrix(tmp_x)[,-1]
            tmp_cvfit<-cv.glmnet(x=tmp_x,y=ytrain,family=tweedie(link.power=0,var.power=1.7),alpha=0.5
                                 ,offset=log(fitted(fit1)),type.measure = 'deviance')
            tmp_lambda<- tmp_cvfit$lambda.min
            tmp_fit<-glmnet(x=tmp_x,y=ytrain,family=tweedie(link.power=0,var.power=1.7),alpha=0.5
                                 ,offset=log(fitted(fit1)),lambda=tmp_lambda)
            tmp_coef<-as.matrix(coef(tmp_fit))[,1]
            if (length(which(tmp_coef!=0))==0){
                result_round1[i,j]=9999}
            else{
                tmp_x<-model.matrix(~autocross[,i]*autocross[,j])
                tmp_fitted<-exp(tmp_x%*%tmp_coef+log(fitted(fit1)))
                result_round1[i,j]<-mean(tweedie.dev(ytrain,tmp_fitted,1.7))
                cat_coef=append(cat_coef,list(as.numeric(tmp_coef)))
                cat_coef_names=c(cat_coef_names,paste(as.character(i),as.character(j),sep=''))       
            }  
            names(cat_coef)=cat_coef_names
    }
  }
}

In [49]:
result_round1_normal=matrix(nrow=dim(autocross)[2],ncol=dim(autocross)[2])
for(i in 2:(dim(autocross)[2])-1){
    for(j in (i+1):(dim(autocross)[2])){
        if (is.numeric(autocross[,i])&&is.numeric(autocross[,j])){
            tmp_x <- autocross[,i]*autocross[,j]
            tmp_fit<-glm(ytrain~tmp_x+offset(log(fitted(fit1)))
                   ,family=tweedie(link.power=0,var.power=1.7),control=list(maxit=500))
            result_round1_normal[i,j]=mean(residuals(tmp_fit)^2)
        }
        else{
            tmp_x<-model.matrix(~autocross[,i]*autocross[,j])
            tmp_x<-as.matrix(tmp_x)[,-1]
            tmp_fit<-glm(ytrain~tmp_x+offset(log(fitted(fit1)))
                   ,family=tweedie(link.power=0,var.power=1.7),control=list(maxit=500))
            result_round1_normal[i,j]=mean(residuals(tmp_fit)^2)
        }
    }
  }

In [50]:
round1_index_normal <- which(result_round1_normal == min(result_round1_normal,na.rm=T), arr.ind = TRUE)
round1_index_normal

row,col
1,3


In [50]:
round1_index <- which(result_round1 == min(result_round1,na.rm=T), arr.ind = TRUE)
round1_index

row,col
1,2


In [27]:
cat_coef$'13'

[1]  0.004041093 -0.079706133 -0.373300365  0.618033499

In [51]:
round1_fit<-glm(y~.+x1*x2,data=design_x_strain
                   ,family=tweedie(link.power=0,var.power=1.7),control=list(maxit=500))
summary(round1_fit)


Call:
glm(formula = y ~ . + x1 * x2, family = tweedie(link.power = 0, 
    var.power = 1.7), data = design_x_strain, control = list(maxit = 500))

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-6.7387  -2.9175  -2.4108  -0.9727  11.8779  

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -0.02114    0.16425  -0.129    0.898    
x12          0.69773    0.14351   4.862 1.24e-06 ***
x2           0.56071    0.11128   5.039 5.02e-07 ***
x3           0.32037    0.06962   4.602 4.40e-06 ***
x4           0.65748    0.06861   9.583  < 2e-16 ***
x12:x2       0.92222    0.13661   6.751 1.82e-11 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Tweedie family taken to be 14.45999)

    Null deviance: 36207  on 2499  degrees of freedom
Residual deviance: 18585  on 2494  degrees of freedom
AIC: NA

Number of Fisher Scoring iterations: 11


In [52]:
mean(residuals(round1_fit)^2)/mean(residuals(fit1)^2)

[1] 0.955757

In [53]:
result_round2=matrix(nrow=dim(autocross)[2],ncol=dim(autocross)[2])
cat_coef_round2=list()
cat_coef_names_round2=c()
for(i in 2:(dim(autocross)[2])-1){
    for(j in (i+1):(dim(autocross)[2])){
        if(i==1 &&j==2){result_round2[i,j]=999999}
        else{
            if (is.numeric(autocross[,i])&&is.numeric(autocross[,j])){
                tmp_x <- autocross[,i]*autocross[,j]
                tmp_fit<-glm(ytrain~tmp_x+offset(log(fitted(round1_fit)))
                       ,family=tweedie(link.power=0,var.power=1.7),control=list(maxit=500))
                result_round2[i,j]=mean(residuals(tmp_fit)^2)
            }
            else {
            tmp_x<-model.matrix(~autocross[,i]*autocross[,j])
            tmp_x<-as.matrix(tmp_x)[,-1]
            tmp_cvfit<-cv.glmnet(x=tmp_x,y=ytrain,family=tweedie(link.power=0,var.power=1.7),alpha=0.5
                                 ,offset=log(fitted(round1_fit)),type.measure = 'deviance')
            tmp_lambda<- tmp_cvfit$lambda.min
            tmp_fit<-glmnet(x=tmp_x,y=ytrain,family=tweedie(link.power=0,var.power=1.7),alpha=0.5
                                 ,offset=log(fitted(round1_fit)),lambda=tmp_lambda)
            tmp_coef<-as.matrix(coef(tmp_fit))[,1]
            if (length(which(tmp_coef!=0))==0){
                result_round2[i,j]=9999}
            else{
                tmp_x<-model.matrix(~autocross[,i]*autocross[,j])
                tmp_fitted<-exp(tmp_x%*%tmp_coef+log(fitted(round1_fit)))
                result_round2[i,j]<-mean(tweedie.dev(ytrain,tmp_fitted,1.7))
                cat_coef_round2=append(cat_coef_round2,list(as.numeric(tmp_coef)))
                cat_coef_names_round2=c(cat_coef_names_round2,paste(as.character(i),as.character(j),sep=''))       
            }  
            names(cat_coef_round2)=cat_coef_names_round2
    }
            }
  }
}

In [54]:
result_round_2_threeway=rep(0,dim(autocross)[2])
cat_coef_round23=list()
cat_coef_names_round23=c()
for(i in 1:(dim(autocross)[2])){
  if (i==1 || i==2){
    result_round_2_threeway[i]=99999
  }
  else{
      tmp_x<-model.matrix(~autocross[,1]*autocross[,2]*autocross[,i])
      tmp_x<-tmp_x[,-1]
            tmp_cvfit<-cv.glmnet(x=tmp_x,y=ytrain,family=tweedie(link.power=0,var.power=1.7),alpha=0.5
                                 ,offset=log(fitted(round1_fit)),type.measure = 'deviance')
            tmp_lambda<- tmp_cvfit$lambda.min
            tmp_fit<-glmnet(x=tmp_x,y=ytrain,family=tweedie(link.power=0,var.power=1.7),alpha=0.5
                                 ,offset=log(fitted(round1_fit)),lambda=tmp_lambda)
            tmp_coef<-as.matrix(coef(tmp_fit))[,1]
            if (length(which(tmp_coef!=0))==0){
                result_round_2_threeway[i]=9999}
            else{
                tmp_x<-model.matrix(~autocross[,1]*autocross[,2]*autocross[,i])
                tmp_fitted<-exp(tmp_x%*%tmp_coef+log(fitted(round1_fit)))
                result_round_2_threeway[i]<-mean(tweedie.dev(ytrain,tmp_fitted,1.7))
                cat_coef_round23=append(cat_coef_round23,list(as.numeric(tmp_coef)))
                cat_coef_names_round23=c(cat_coef_names_round23,paste(as.character(i),sep=''))       
            }  
            names(cat_coef_round23)=cat_coef_names_round23
}
    }

In [56]:
result_round2_final<-cbind(result_round2,result_round_2_threeway)
round2_index<-which(result_round2_final == min(result_round2_final,na.rm=T), arr.ind = TRUE)
round2_index

row,col
4,5


In [58]:
head(result_round2_final)

,,,,result_round_2_threeway
NA,999999,7.433906,7.409127,99999.000000
NA,NA,7.428986,7.314837,99999.000000
NA,NA,NA,7.433906,7.433906
NA,NA,NA,NA,7.035157


In [59]:
round2_fit<-glm(y~.+x1*x2*x4,data=design_x_strain
                   ,family=tweedie(link.power=0,var.power=1.7),control=list(maxit=500))
summary(round2_fit)


Call:
glm(formula = y ~ . + x1 * x2 * x4, family = tweedie(link.power = 0, 
    var.power = 1.7), data = design_x_strain, control = list(maxit = 500))

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-8.4205  -2.7855  -2.4277  -0.8794  11.7444  

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.26859    0.17005   1.579  0.11436    
x12          0.30777    0.18881   1.630  0.10321    
x2           0.27333    0.15059   1.815  0.06963 .  
x3           0.30693    0.05944   5.164 2.61e-07 ***
x4           0.31626    0.10548   2.998  0.00274 ** 
x12:x2       0.44195    0.18675   2.367  0.01803 *  
x12:x4       0.21599    0.12688   1.702  0.08881 .  
x2:x4        0.31756    0.09971   3.185  0.00147 ** 
x12:x2:x4    0.23217    0.11727   1.980  0.04784 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Tweedie family taken to be 10.11432)

    Null deviance: 36207  on 2499  degrees of freedom
Res

In [61]:
mean(residuals(round2_fit)^2)/mean(residuals(round1_fit)^2)

[1] 0.9462817

In [62]:
result_round3=matrix(nrow=dim(autocross)[2],ncol=dim(autocross)[2])
for(i in 2:(dim(autocross)[2])-1){
    for(j in (i+1):(dim(autocross)[2])){
        if(i==1 &&j==2){result_round3[i,j]=999999}
        else{
            if (is.numeric(autocross[,i])&&is.numeric(autocross[,j])){
                tmp_x <- autocross[,i]*autocross[,j]
                tmp_fit<-glm(ytrain~tmp_x+offset(log(fitted(round2_fit)))
                       ,family=tweedie(link.power=0,var.power=1.7),control=list(maxit=500))
                result_round3[i,j]=mean(residuals(tmp_fit)^2)
            }
            else {
            tmp_x<-model.matrix(~autocross[,i]*autocross[,j])
            tmp_x<-as.matrix(tmp_x)[,-1]
            tmp_cvfit<-cv.glmnet(x=tmp_x,y=ytrain,family=tweedie(link.power=0,var.power=1.7),alpha=0.5
                                 ,offset=log(fitted(round2_fit)),type.measure = 'deviance')
            tmp_lambda<- tmp_cvfit$lambda.min
            tmp_fit<-glmnet(x=tmp_x,y=ytrain,family=tweedie(link.power=0,var.power=1.7),alpha=0.5
                                 ,offset=log(fitted(round2_fit)),lambda=tmp_lambda)
            tmp_coef<-as.matrix(coef(tmp_fit))[,1]
            if (length(which(tmp_coef!=0))==0){
                result_round3[i,j]=9999}
            else{
                tmp_x<-model.matrix(~autocross[,i]*autocross[,j])
                tmp_fitted<-exp(tmp_x%*%tmp_coef+log(fitted(round2_fit)))
                result_round3[i,j]<-mean(tweedie.dev(ytrain,tmp_fitted,1.7))     
            }  

    }
            }
  }
}

In [63]:
result_round_3_threeway=rep(0,dim(autocross)[2])
for(i in 1:(dim(autocross)[2])){
  if (i==1 || i==2||i==4){
    result_round_3_threeway[i]=99999
  }
  else{
      tmp_x<-model.matrix(~autocross[,1]*autocross[,2]*autocross[,i])
      tmp_x<-tmp_x[,-1]
            tmp_cvfit<-cv.glmnet(x=tmp_x,y=ytrain,family=tweedie(link.power=0,var.power=1.7),alpha=0.5
                                 ,offset=log(fitted(round2_fit)),type.measure = 'deviance')
            tmp_lambda<- tmp_cvfit$lambda.min
            tmp_fit<-glmnet(x=tmp_x,y=ytrain,family=tweedie(link.power=0,var.power=1.7),alpha=0.5
                                 ,offset=log(fitted(round2_fit)),lambda=tmp_lambda)
            tmp_coef<-as.matrix(coef(tmp_fit))[,1]
            if (length(which(tmp_coef!=0))==0){
                result_round_2_threeway[i]=9999}
            else{
                tmp_x<-model.matrix(~autocross[,1]*autocross[,2]*autocross[,i])
                tmp_fitted<-exp(tmp_x%*%tmp_coef+log(fitted(round2_fit)))
                result_round_3_threeway[i]<-mean(tweedie.dev(ytrain,tmp_fitted,1.7))   
            }  

}
    }

In [64]:
tmp_x<-model.matrix(~autocross[,1]*autocross[,2]*autocross[,4]*autocross[,3])
tmp_x<-tmp_x[,-1]
tmp_cvfit<-cv.glmnet(x=tmp_x,y=ytrain,family=tweedie(link.power=0,var.power=1.7),alpha=0.5
                                 ,offset=log(fitted(round2_fit)),type.measure = 'deviance')
tmp_lambda<- tmp_cvfit$lambda.min
tmp_fit<-glmnet(x=tmp_x,y=ytrain,family=tweedie(link.power=0,var.power=1.7),alpha=0.5
                                 ,offset=log(fitted(round2_fit)),lambda=tmp_lambda)
tmp_coef<-as.matrix(coef(tmp_fit))[,1]
tmp_x<-model.matrix(~autocross[,1]*autocross[,2]*autocross[,4]*autocross[,3])
tmp_fitted<-exp(tmp_x%*%tmp_coef+log(fitted(round2_fit)))
result_round_3_fourway<-mean(tweedie.dev(ytrain,tmp_fitted,1.7))   

In [65]:
result_round3_final<-cbind(result_round3,result_round_3_threeway)
round3_index<-which(result_round3_final == min(result_round3_final,na.rm=T), arr.ind = TRUE)
round3_index

row,col
3,4


In [66]:
min(result_round3_final,na.rm=T)
result_round_3_fourway

[1] 7.030408

[1] 7.034569

In [67]:
round3_fit<-glm(y~.+x1*x2*x4+x3*x4,data=design_x_strain
                   ,family=tweedie(link.power=0,var.power=1.7),control=list(maxit=500))
summary(round3_fit)


Call:
glm(formula = y ~ . + x1 * x2 * x4 + x3 * x4, family = tweedie(link.power = 0, 
    var.power = 1.7), data = design_x_strain, control = list(maxit = 500))

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-8.0039  -2.7882  -2.4200  -0.8954  11.6292  

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.42577    0.18681   2.279  0.02275 *  
x12          0.30178    0.18854   1.601  0.10959    
x2           0.26515    0.15076   1.759  0.07875 .  
x3           0.44351    0.09422   4.707 2.65e-06 ***
x4           0.21046    0.11801   1.783  0.07464 .  
x12:x2       0.43299    0.18654   2.321  0.02036 *  
x12:x4       0.21799    0.12713   1.715  0.08652 .  
x2:x4        0.32665    0.10036   3.255  0.00115 ** 
x3:x4       -0.10788    0.05762  -1.872  0.06130 .  
x12:x2:x4    0.24302    0.11764   2.066  0.03895 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Tweedie family taken to be 10.00

In [68]:
mean(residuals(round3_fit)^2)/mean(residuals(round2_fit)^2)

[1] 0.9982143

In [45]:
tmp_x<- model.matrix(~.+x1*x2*x3,data=design_x_stest[,1:5])
tmp_x=tmp_x[,colnames(tmp_x)!='x12:x22']
tmp_x=tmp_x[,colnames(tmp_x)!='x12:x23']
tmp_x=tmp_x[,colnames(tmp_x)!='x22:x3']
tmp_x=tmp_x[,colnames(tmp_x)!='x12:x23:x3']
predd_round2<-exp(tmp_x%*%round2_fitt$coefficients)

In [46]:
mean(tweedie.dev(design_x_stest$y,pred_fit1,1.7))
mean(tweedie.dev(design_x_stest$y,pred_round1,1.7))
mean(tweedie.dev(design_x_stest$y,pred_round2,1.7))
mean(tweedie.dev(design_x_stest$y,predd_round2,1.7))

[1] 1.627649

[1] 1.603767

[1] 1.594172

[1] 1.596013